<a href="https://colab.research.google.com/github/RahulXavier/HEXA/blob/master/HEXA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#For authentication from google drive and to link googe drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#1. Get sample file
#2. Read image and display
from IPython.display import Image
Image("/content/drive/My Drive/Colab Notebooks/Project-Sign samples/HEXA.jpg")

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

# This cell contains all the references to libraries that are needed to train a convolutional neural network

from __future__ import print_function, division  
#To support Py2.0's print function  #To correct ambiguous division operator

import numpy as np   #for working with arrays
import random #to give a random number between 0 and 1
import os #The OS module in Python provides functions for creating and removing a directory (folder), fetching its contents, changing and identifying the current directory, etc.
import glob #To find files with a certain extension in a library. Eg: To find .jpg files

# https://opencv.org/
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2 #OpenCV with Numpy Support
import datetime
import pandas as pd #For data analysis
import time 
import h5py #To work with HDF5 format, which stores huge amounts of numerical data #https://www.h5py.org/ to store trained file
import csv #To handle CSV files (Excel)

# https://docs.scipy.org/doc/scipy-1.2.1/reference/generated/scipy.misc.imresize.html
from skimage.transform import resize

#from sklearn.cross_validation import KFold, train_test_split #To estimate skill of machine learning model #Split to Training and Testing, to avoid overfitting
from sklearn.metrics import log_loss, confusion_matrix #To calculate loss function #Matrix to visualize performance
from sklearn.utils import shuffle #shuffle data for better training

from PIL import Image, ImageChops, ImageOps  #To work with images #For image editing #For automatic filters on images
#PIL = Python Imaging Library

import matplotlib.pyplot as plt #To replicate plot function in matlab

from keras import backend as K #To choose backend(Tensorflow,Theano,CTNK,etc)
from keras.callbacks import EarlyStopping, Callback #To stop training epochs if training stops improving, to prevent overfitting #To view staticstics during training
from keras.utils import np_utils #Embed NumPy with Keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array #For data augmentation(rotate,resize,shearing etc) #Load image as PIL #To make 3D NumPy array of img
from keras import optimizers #Algorithm to update weights
from keras.models import Sequential, model_from_json #For layer-by-layer model creation #A type of ANN model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Activation, Dropout, Flatten, Dense #layers in CNN

In [ ]:
# Importing the Keras libraries and packages
from keras.models import Sequential #import sequential model(layer-by-layer) as comapared to functional model
from keras.layers import Conv2D #For convolution on images
from keras.layers import MaxPooling2D #For max-poooling on @d feature maps
from keras.layers import Flatten #To Convert pooled 2D image to 1D array
from keras.layers import Dense #Connect input to every output

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
# make 32 feature detectors with a size of 3x3
# choose the input-image's format to be 64x64 with 3 channels     #relu = Rectified Linear Unit
classifier.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation="relu"))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2))) #Converts 2*2=4 pixels to 1

# Adding a second convolutional layer and second pooling layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(activation="softmax", units=128))
#classifier.add(Dense(activation="sigmoid", units=8))  #Set number of output nodes here
classifier.add(Dense(activation="softmax", units=8))

# Compiling the CNN
#Optimizer deals with weight updation using loss function. https://medium.com/datadriveninvestor/optimizers-for-training-neural-networks-e0196662e21e
#Adam = Adaptive Momentum Estimation
#Loss function required to estimate loss https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
#Metrics are used to evaluate model #'accuracy' is to display accuracy while training
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# use ImageDataGenerator to preprocess the data
from keras.preprocessing.image import ImageDataGenerator

# Augment the data that we have to create more training data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=10,
                                   shear_range = 0,
                                   zoom_range = 0)

test_datagen = ImageDataGenerator(rescale = 1./255)


batchsize = 32 #Default 32
# Prepare training data
# Select path, Input resolution, input resolution, Batch size, class number.
# The batch size is a number of samples processed before the model is updated. https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network
# class_mode selection - https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720
training_data = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/Project-Sign samples/Signature Datasets/Training',
                                                 target_size = (128, 128),
                                                 batch_size = batchsize,
                                                 class_mode = 'categorical')
# prepare test data
test_data = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/Project-Sign samples/Signature Datasets/Testing',
                                            target_size = (128, 128),
                                            batch_size = batchsize,
                                            class_mode = 'categorical')

# finally start computation
# to improve the model accuracy you can increase the number of steps_per_epoch to e.g. 8000
# increase the number of epochs to 5-25
# increase the validation steps
# this parametters allow for the model to optimize
wandb.init(project="CNN")
classifier.fit_generator(training_data,
                         steps_per_epoch = (800 / batchsize),  #real 4000
                         epochs = 20,
                         validation_data = test_data,
                         validation_steps = 20,
                         callbacks=[WandbCallback()])







Found 696 images belonging to 8 classes.
Found 120 images belonging to 8 classes.


<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





25/25 [==============================] - 241s 10s/step - loss: 2.0211 - acc: 0.2550 - val_loss: 1.9611 - val_acc: 0.3750
Epoch 2/20
25/25 [==============================] - 6s 234ms/step - loss: 1.9410 - acc: 0.4096 - val_loss: 1.9166 - val_acc: 0.5000
Epoch 3/20
25/25 [==============================] - 5s 214ms/step - loss: 1.8877 - acc: 0.5709 - val_loss: 1.8633 - val_acc: 0.6167
Epoch 4/20
25/25 [==============================] - 6s 234ms/step - loss: 1.8448 - acc: 0.6517 - val_loss: 1.8231 - val_acc: 0.7167
Epoch 5/20
25/25 [==============================] - 6s 234ms/step - loss: 1.8011 - acc: 0.7512 - val_loss: 1.7871 - val_acc: 0.7250
Epoch 6/20
25/25 [==============================] - 5s 219ms/step - loss: 1.7676 - acc: 0.7125 - val_loss: 1.7530 - val_acc: 0.7250
Epoch 7/20
25/25 [==============================] - 6s 224ms/step - loss: 1.7330 - acc: 0.7383 - val_loss:

In [ ]:
#To save training data
#To save whole data - classifier.save(filepath) , To save just weights without architecture - classifier.save_weights(filepath)
os.chdir("/content/drive/My Drive/Colab Notebooks/Project-Sign samples/")
#classifier.save_model('location') , add architecture seperately
classifier.save('model1.h5py')

In [ ]:
#To load training data
from keras.models import load_model
classifier = load_model('/content/drive/My Drive/Colab Notebooks/Project-Sign samples/model1.h5py')

In [ ]:
#Model testing

from IPython.display import Image #Notebook function to display media, here Image #https://www.dev2qa.com/how-to-display-rich-output-media-audio-video-image-etc-in-ipython-jupyter-notebook/
#Image('/content/drive/Colab Notebooks/Project-Sign samples/Input images/Reenu.jpg') #To display image

# to make predictions
import numpy as np #To convert image to no array
from keras.preprocessing import image #To work with images(load,save,etc)
img_num = input('Enter Image Number: ')
input_image = '/content/drive/My Drive/Colab Notebooks/Project-Sign samples/Input images/'+img_num+'.jpg'

test_image = image.load_img(input_image, target_size = (128, 128)) #get image with predefined resolution
test_image = image.img_to_array(test_image) #Convert image to np array
test_image = np.expand_dims(test_image, axis = 0) #To change dimension of image, create 1D array

result = classifier.predict(test_image) #Output predicted
result = result[0]  #Takes 1st o/p from numpy array
result = list(result) #Convert numpy object to list
print(result)
namelist = ['Reenu','Renie','Sandra Fulgence','Sholia','Shwetha Sandeep','Sweta Satish','Tabitha','Vivek SP']
index = result.index(max(result))
print('The signature belongs to',namelist[index])

imagename = '/content/drive/My Drive/Colab Notebooks/Project-Sign samples/Output Photos/' + str(index) + '.jpg'


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os, sys
from PIL import Image
from glob import glob
img1=mpimg.imread(input_image)
img2=mpimg.imread(imagename)
plt.figure(1)
plt.subplot(121)
plt.imshow(img1)
plt.subplot(122)
plt.imshow(img2)
plt.show()

In [ ]:
#To convert image to black and white
import cv2
from google.colab.patches import cv2_imshow
index = input('Enter image number to convert to B&W: ')
  
originalImage = cv2.imread('/content/drive/My Drive/Colab Notebooks/Project-Sign samples/Input images/'+index+'.jpg',0)
  
(thresh, blackAndWhiteImage) = cv2.threshold(originalImage, 127, 255, cv2.THRESH_BINARY)
cv2_imshow(blackAndWhiteImage)

cv2_imshow(originalImage)
cv2.imwrite('/content/drive/My Drive/Colab Notebooks/Project-Sign samples/Input images/'+index+'.jpg', blackAndWhiteImage)